In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# P1 Calibration and False Alarm Control\n",
        "\n",
        "Demonstrates fitting a null threshold and inspecting alert rate."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "\n",
        "from ssp.core.config import SyntheticConfig\n",
        "from ssp.datasets.synthetic import generate_synthetic\n",
        "from ssp.p1_detect.score_nonml import score_nonml_windows\n",
        "from ssp.p1_detect.calibrate import CalibrateSpec, ScoreCalibrator\n",
        "\n",
        "cfg = SyntheticConfig(seed=2, n_sensors=6, n_steps=10000, dt_s=0.01, anomaly_rate=0.0)\n",
        "ds = generate_synthetic(cfg)\n",
        "\n",
        "d = int(np.max(ds.sensor_id) + 1)\n",
        "y_mv = np.zeros((cfg.n_steps, d), dtype=np.float64)\n",
        "y_mv[ds.step_index, ds.sensor_id] = ds.y\n",
        "\n",
        "win = 64\n",
        "n_win = cfg.n_steps - win + 1\n",
        "x = np.empty((n_win, win, d), dtype=np.float64)\n",
        "for j in range(d):\n",
        "    x[:, :, j] = np.lib.stride_tricks.sliding_window_view(y_mv[:, j], win)\n",
        "\n",
        "s = score_nonml_windows(x)\n",
        "cal = ScoreCalibrator(CalibrateSpec(target_fa_rate=1e-3))\n",
        "cal.fit_null(s)\n",
        "thr = cal.threshold()\n",
        "fa = float(np.mean(s >= thr))\n",
        "\n",
        "print('threshold:', thr)\n",
        "print('empirical FA rate:', fa)\n"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.11"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}
